In [1]:
import numpy as np
import random

import numpy as np
import random


USE_WIDGETS = True



if USE_WIDGETS:
    %matplotlib widget
else:
    %matplotlib inline
    
import matplotlib as mpl

import matplotlib.pyplot as plt



plt.rcParams.update({
  "text.usetex": True,
  "font.family": "Helvetica",
  "figure.max_open_warning" : 1000,
  "font.size": 10
})


In [2]:
def GenerateState(N = 3):
    value_dict = {}
    positions = {}
    array = np.zeros((N,N)).astype(int)
    
    for i in range(N**2):
        value_dict[i+1] =i+1
    
    
    for i in range(N):
        for j in range(N):
            key = random.choice([*value_dict])
            array[i, j] = value_dict[key]
            positions[key] = (i, j)
            del value_dict[key]
         
    
    return (array, positions)

In [3]:
def Fitness(state):
    N = len(state[0])
    array = state[0]
    M = int(N*(N**2 +1)*0.5)
    
    
    f = 0
    
    d1 = 0
    d2 = 0
    for i in range(N):
        d1 += array[i, i]
        d2 += array[i, N-i-1]
        
        f += np.abs(np.sum(array[:, i]) - M)
        f += np.abs(np.sum(array[i, :]) - M)
    
    f += np.abs(d1 - M)
    f += np.abs(d2 - M)
              
    
    return int(f)
    
    

In [4]:
def Neighbour(state, k = 1):
    N = len(state[0])
    
    
    MAX = N**2
    
    if k > MAX:
        k = MAX
    if k < 1:
        k = 1
        
    keys = []
    
    count = 0
    lst = []
    
    #Idemo po modulu N tako da je dovoljno samo +1 susedstvo da proveravamp
    # N-1-2, a iz 2 mozemo u 1-2-3, itd... pokricemo sve slucajeve samo sa +1
    value_dict = state[1].copy()
    while count != k:
        key = random.choice([*value_dict])
        keys.append(value_dict[key])
        del value_dict[key]
        count+=1
       
    for index in keys:
        pos = state[1].copy()
        array = np.array(state[0])
            
        key = array[index] + 1
        last = key - 1
        
        if key > MAX:
            key = 1
        
        #print('swap', index, '->', pos[key], 'key', key, 'N', N)
        array[index], array[pos[key]] = array[pos[key]], array[index]
                    
        pos[key], pos[last] = index, pos[key]
        
        
        lst.append((array, pos))
        
        
        
        count+=1
    return lst
    
    

In [5]:
def RandomSearch(max_iter = 100, N = 3):
    TOTAL = 0
    cnt = 0
    
    MIN = float('inf')
    best = np.zeros((N, N))
    for i in range(max_iter):
        state = GenerateState(N)
        f = Fitness(state)
        TOTAL += f
        cnt+=1
        
        if f < MIN:
            MIN = f
            best = state
            
            if f == 0:
                TOTAL = TOTAL/cnt
                return best, f 
            
    
    return best, f
            

In [6]:
def GreedySearch(starting_state, total_iter = 100, neighb_num = 6):
    current = starting_state
    
    N = len(starting_state[0])
    history = []
    best = 0 
    
    if neighb_num >  N**2:
        k = N**2 
    else:
        if neighb_num < 2:
            k = 2
        else:
            k = neighb_num  
        
    for i in range(total_iter):
        MIN = float('inf')
        
        for state in Neighbour(current, k):
            f = Fitness(state)
            if f < MIN:
                MIN = f
                best = state
                
                if f == 0:
                    history.append(f)
                    return best, history    
                      
        
        history.append(MIN)  
        
        current = best
        
    
    return best, history        


In [7]:
def SimulatedAnnealing(starting_state, T_start, Mk, beta = 0.5, max_iter = 10000 ,neighb_num = 6):
    current = starting_state
    N = len(current[0])
    history = []
    
    fo = Fitness(current)
    t = T_start
    eps = -1/(N**3)/np.log(0.2)
    #print(eps)
    GLOB_MIN = fo
    glob_best = current
    count = 0
    beta_t = beta
    
    
    
    while t > eps and count < max_iter:
            
        best = current
        p = []
        for m in range(Mk):
            for new in Neighbour(current, neighb_num):
                fn = Fitness(new)
                count+=1
            
                delta = fn - fo
            
                if delta <= 0:
                    best = new
                    fo = fn
                    
                    if fn < GLOB_MIN:
                        glob_best = new
                        GLOB_MIN = fn
                        
                    #print(fo, delta)
                else:
                    P = np.exp(-delta/t)
                    p.append(P)
                    #print(P, t, delta)
                
                    if np.random.rand() < P:
                        best = new
                        fo = fn
                        #print(fo, delta)
                        
                if fo == 0:
                    history.append((fo, t, p))
                    return glob_best, history        
            history.append((fo, t, p))
            current = best 

            
            
        t = t*beta

    
    return glob_best, history        

In [8]:
def BeamSearch(starting_states, max_iter = 3000, num_states = 4, neighb_num = 12):
    states = starting_states
    #for i in range(num_states):
    #    states.append(GenerateState(N))
    #    
    count = 0
    candidates = []
    history = []
    
    MIN = float('inf')
    best = None
    candidates = [1]* neighb_num * num_states
    while count < max_iter:
       
        idx = 0
        for state in states:
            for cand in Neighbour(state, k = neighb_num):
                candidates[idx] = (cand, Fitness(cand))
                idx+= 1
            #candidates.append((state, Fitness(state)))    
        candidates = sorted(candidates, key=lambda x: x[1])
        data = [candidates[i][1] for i in range(len(candidates))]
        history.append(data)
        #print(candidates)
        
        
        for i in range(num_states):
            states[i] = candidates[i][0]
            if candidates[i][1] < MIN:
                MIN = candidates[i][1]
                best = candidates[i][0]
                
                if MIN == 0:
                    return best, history
        #states[-1] = candidates[-1][0]
        #if candidates[-1][1] < MIN:
        #        MIN = candidates[-1][1]
        #        best = candidates[-1][0]
        #        
        #        if MIN == 0:
        #            return best
        count+=1
    
    
    return best, history

In [9]:
def GenethicAlgorithm(starting_states, max_iter = 1000, pop_size = 10,elitism = 3, k = 4, mutation_rate = 0.01, alpha = 0.1,crossover_prob = 0.6,sub_pop = 10,migration_interval = 10, migration_rate = 0.2, rate_increase = 0.01,  depth = 3, search_size =4, search_interval = 10):

    

    MAX_RATE = 0.8
    if pop_size != len(starting_states):
        pop_size = len(starting_states)
        
    #selection_size = pop_size - 1
    #migration_interval
    #migration_rate
    #rate_increase
        
    if k < 1:
        k = 2

    if mutation_rate < 0:
        mutation_rate = 0
    population = [(0, float('inf'))]*pop_size
    idx = 0
   

    history = []
    if not starting_states:
        return
    
    for state in starting_states:
        population[idx] = (state, Fitness(state))
        idx+=1
    
    
    sub_populations = [[] for _ in range(sub_pop)]
    idx = 0
    
    while idx!= pop_size:
        sub_populations[idx%sub_pop].append(population[idx])
        idx+=1
    
    
    
   
    generation = 0
    
    old_mutation_rate = mutation_rate
    MIN = float('inf')
    best = None
    
    old_fi = [0]*sub_pop
    new_fi = [0]*sub_pop
    
    
    
    
    best_so_far = [float('inf')]*sub_pop
    mutation_rates = [mutation_rate]*sub_pop
    migration_rates = [migration_rate]*sub_pop
    stagnation = [0]*sub_pop
    search_cost = 0
    m = len(sub_populations[0])

    while generation <= max_iter - search_cost//m:
        
        data = []
        for idx, population in enumerate(sub_populations):
            #print(len(population))
            population.sort(key = lambda x: x[1])
            pop_size = len(population)
            selection_size = len(population)
            new_generation = [None]*(pop_size + selection_size - elitism)
            
            if population[0][1] < MIN:
                MIN = population[0][1]
                best = population[0][0]
            
            if population[0][1] < best_so_far[idx]:
                best_so_far[idx] = population[0][1]
                mutation_rates[idx] = mutation_rate
                stagnation[idx] = 0
            if population[0][1] == best_so_far[idx]:
                #population[0:] = Mutation(population, alpha)
                #mutation_rates[idx] += 0.01, depth, search_size 
                stagnation[idx] += 1
                if stagnation[idx] == search_interval:
                    population[:] = Search(population, search_size , depth)
                    stagnation[idx] = 0
                    search_cost+=search_size*depth
                #if mutation_rates[idx] > alpha:
                #    mutation_rates[idx] = alpha
                
            
            for i in range(pop_size):
                if population[i][1]!=float('inf'):
                    data.append(population[i][1])
            if population[0][1] == 0:
                history.append(data)
                return population[0][0], history



            parents = Selection(population, k, selection_size)

            children= Crossover(parents, crossover_prob)


            mutated = Mutation(children, mutation_rates[idx])
            
            mutated.sort(key = lambda x: x[1])
            population.sort(key = lambda x: x[1])
            new_generation = population[elitism:] +   mutated

            population[elitism:] = random.sample(new_generation, pop_size-elitism) 
           
            
            #if elitism == 0:
            #     population[0:-1] = mutated
            #else:
            #    population[elitism:] = mutated[0 : -elitism]
                    
                    
            population.sort(key = lambda x: x[1])
            if generation%migration_interval == 1 :
                new_fi[idx] = population[0][1]
                
        generation+=1
        #print(generation, max_iter)    
        history.append(data)
        if generation%migration_interval == 0 :
            
            for idx, population in enumerate(sub_populations):
                new_fi[idx] = new_fi[idx] - population[0][1]
                if new_fi[idx] > old_fi[idx]:
                    migration_rates[idx] += rate_increase
                    if migration_rates[idx]> 0.8:
                        migration_rates[idx] = 0.8
                    #migration_rate+=rate_increase
                if new_fi[idx] < old_fi[idx]:
                    migration_rates[idx] -= rate_increase
                    if migration_rates[idx] < 0.1:
                        migration_rates[idx] = 0.1
                old_fi[idx] = new_fi[idx]
            
            if migration_rate > 0.8:
                migration_rate = 0.8
            if migration_rate < 0.1:
                migration_rate = 0.1
            
            #print(np.round(migration_rates, 3))
            #print(sub_populations)
                  
            for idx, population in enumerate(sub_populations): 
                to_migrate = int(np.round(len(population)*migration_rates[(idx-1)%sub_pop]))
                
                if to_migrate == 0:
                    continue
                
                #migrators = population[:to_migrate]
                migrators = random.sample(population, to_migrate) 
                sub_populations[(idx+1)%sub_pop][-to_migrate:] = migrators
                #sub_populations[(idx+1)%sub_pop].sort(key = lambda x: x[1])
                
                
                
                
                
                    
            
                
            
        
    return best, history
            
    
    
    
import random

def Tournament(population, k):
    tournament = random.sample(population, k)
    tournament.sort(key=lambda x: x[1], reverse=False)
    
    total = 0
    for i in tournament:
        total+= i[1]
    winners = [0,0]
    idx = 0
    
    #while idx!=2:
    #   for participant in tournament:
    #       prob = 1 - participant[1]/total
    #        if np.random.rand() < prob:
    #            winners[idx] = participant
    #            idx+=1
    #            if idx == 2:
    #                break
    winners = tournament[0:2]
    return winners

def Selection(population, k, selection_size):
    parents = [None]*(selection_size//2)
    for i in range(selection_size//2):
        parent = Tournament(population, k)
        parents[i] = parent
    return parents
# def Selection(population, k, selection_size):
#     voting_candidates = [[None] * 2 for _ in range(selection_size//2)]
    
#     #print(len(voting_candidates))

#     picked = 0
#     idx = 0
#     total = 0
#     while idx < selection_size:
#         total+=population[idx][1]
#         idx+=1


#     idx = 0

#     while picked < selection_size//2:
#         j = 0
#         while j < 2:

#             prob = 1 - population[idx][1]/total
#             if np.random.rand() < prob:
#                 voting_candidates[picked][j] = population[idx]
#                 j+=1

#             idx = (idx + 1)%selection_size



#         picked+=1
        
#     #print(voting_candidates)

#     return voting_candidates
    
def Crossover(parents, crossover_prob):
    dim = len(parents[0][0][0][0])
    MAX = dim**2
    
    #print(parents)
    #print(len(parents))
    children = [None]*len(parents)*2
    
    removed = []
    idx = 0
    for pair in parents:
        
        child2 = (pair[1][0][0].copy() ,pair[1][0][1].copy())
        child1 = (pair[0][0][0].copy() ,pair[0][0][1].copy())
        
        if np.random.rand() < crossover_prob:
            k = np.random.randint(1, MAX)
            value_dict = pair[1][0][1].copy()
            count = 0
            keys = []
            while count != k:
                key = random.choice([*value_dict])
                keys.append(key)
                del value_dict[key]
                count+=1
            #cxpoint1 = np.random.randint(0, MAX - 1)
            #cxpoint2 = np.random.randint(cxpoint1 + 1, MAX)

            #print(cxpoint1,cxpoint2)

            #print(pair[0][0][0].flatten())
            to_be_crossed_1 = [None]*len(keys)
            to_be_crossed_2 = [None]*len(keys)
            IDX = 0
            #print(keys)
            for key in keys:
                to_be_crossed_1[IDX] = pair[0][0][0][pair[0][0][1][key]]
                #print(to_be_crossed_1)
                to_be_crossed_2[IDX] = pair[1][0][0][pair[0][0][1][key]]
                IDX+=1
            #print(to_be_crossed_1, to_be_crossed_2)
            #print(to_be_corssed_1)
            for value in to_be_crossed_1:
                #print(value)
                new_pos = child1[1][value]
                old_pos = child2[1][value]
                old_value = child2[0][new_pos]

                child2[0][old_pos], child2[0][new_pos] = \
                child2[0][new_pos], child2[0][old_pos]

                child2[1][value], child2[1][old_value] = \
                new_pos, old_pos

            for value in to_be_crossed_2:
                new_pos = pair[1][0][1][value]
                old_pos = child1[1][value]
                old_value =child1[0][new_pos]

                child1[0][old_pos], child1[0][new_pos] = \
                child1[0][new_pos], child1[0][old_pos]

                child1[1][value], child1[1][old_value] = \
                new_pos, old_pos
        
        #print(child1, child2)
        children[idx] = (child1, Fitness(child1))    
        idx+=1
        children[idx] = (child2, Fitness(child2)) 
        idx+=1
        
        
    #print(children)
    #print(len(children))
    return children
            
            
        
        
    
    
    
    
    #print(voters)
    #     N = len(voters[0][0][0])
    #     value_dict = {}
    #     positions = {}
    #     array = np.zeros((N,N)).astype(int) - 1
    #     votes = {}
    #     visited = {}
    
    
  
    
    
    #     for i in range(N**2):
    #         value_dict[i+1] =i+1
    #         votes[i+1] = 0
    #         visited[i+1] = False


    #     voted = [None]*p
    #     for i in range(N):
    #         for j in range(N):
    #             idx = 0

    #             for chromosome in voters:
    #                 votes[chromosome[0][0][i, j]]+=1
    #                 voted[idx] = chromosome[0][0][i, j]
    #                 idx+=1

    #             #print(votes)

    #             MAX = -float('inf')
    #             best = -1
    #             for vote in voted:      
    #                 if votes[vote] > MAX and not visited[vote]:
    #                     best = vote
    #                     MAX = votes[vote]

    #             if best!=-1:
    #                 if best >= p:
    #                     array[i, j] = best
    #                     positions[best] = (i, j)
    #                     del value_dict[best]
    #                     visited[best] = True


    #             for vote in voted:                      
    #                 votes[vote] = 0   




    #     for i in range(N):
    #         for j in range(N):
    #             if array[i, j] == -1:
    #                 key = random.choice([*value_dict])
    #                 array[i, j] = value_dict[key]
    #                 positions[key] = (i, j)
    #                 del value_dict[key]
            
        
    #state = (array, positions)
            
    #return (state, Fitness(state))

def Mutation(children, mutation_rate):
    dim = len(children[0][0][0])
    MAX = dim**2
    

    end = len(children)
    for idx in range(end): 
        mutated = False
        for i in range(dim):
            for j in range(dim):
                
                if np.random.rand() <  mutation_rate:
                    m = int(np.ceil(np.random.rand()*dim)) - 1
                    n = int(np.ceil(np.random.rand()*dim)) - 1
                    index = (i, j)
                    last = (m, n)
                    if index == last:
                        continue
                    l_k = children[idx][0][0][last]
                    i_k = children[idx][0][0][index]


                    children[idx][0][0][index], children[idx][0][0][last] = \
                    children[idx][0][0][last], children[idx][0][0][index]

                    children[idx][0][1][i_k], children[idx][0][1][l_k] = \
                    last, index
                    mutated = True
                    
                    children[idx] = (children[idx][0], Fitness(children[idx][0]))
                    
            if mutated: break

    return children    
       
def Search(population, search_size, depth):
        idx = 0
        total = 0
        while idx < len(population):
            total+=population[idx][1]
            idx+=1
            
            
        found = False
        idx = 0
        while not found:
            prob = 1 - population[idx][1]/total
            
            #if np.random.rand() < prob:
            if np.random.rand() < prob:
                    found = True
                    #temp = GreedySearch(population[0][0], total_iter = depth, neighb_num = search_size)[0]
                    temp,_ =  SimulatedAnnealing(population[idx][0], T_start = 1, Mk = search_size, beta = 0.95, max_iter = search_size*depth ,neighb_num = 1)
                    f = Fitness(temp)
                    
            idx += 1
        population[idx-1] = (temp, f)
    
        
        
    
        return population
    
        

In [10]:
m = 10
N = 9
neighb = 32
max_iter = 800
histories = [None]*m
for i in range(m):
    s = GenerateState(N)
    _, temp = GreedySearch(s, max_iter, neighb)
    histories[i] = temp
    


In [11]:
M = len(max(histories,key=len))

to_plot = [[] for _ in range(M)]

for i in range(M):
    for _, item in enumerate(histories):
        if len(item) <= i:
            continue

        to_plot[i].append(item[i])
        
average = np.zeros((M,))
std = np.zeros((M,))
MIN = np.zeros((M,))
for idx, item in enumerate(to_plot):
    MIN[idx] = np.min(item)
    average[idx] = np.average(item)
    std[idx] = np.std(item)
    
    

In [12]:
import matplotlib.pyplot as plt

z = 1.96
lower_bound = average - z * std
upper_bound = average + z * std
t = np.linspace(1, M, M)

fig, ax = plt.subplots(1, 1)
ax.plot(t, average, label = 'mean')
ax.fill_between(t, lower_bound, upper_bound, color='gray', alpha=0.2, label='c. interval, 95\%')
ax.grid('on')

ax.set_xlabel('iter')
ax.set_ylabel('Fitness function')
ax.set_title(f"Greedy Algorthm, neighb: {neighb}, matrix size: {N}")

plt.legend()
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
print(np.min(MIN))

0.0


In [14]:
m = 10
N = 9
neighb = 1
beta = 0.95
scailing = 1
Mk = 400
max_iter = 16000
T = 2

histories = [None]*m
for i in range(m):
    s = GenerateState(N)
    _, temp = SimulatedAnnealing(s, T, Mk, beta , max_iter ,neighb_num = neighb)
    histories[i] = temp

In [15]:
#print(histories[0])
#print(max_iter)
#print(histories)
M = len(max(histories,key=len))

to_plot = [[] for _ in range(M)]
prob = [[] for _ in range(M)]
temp = np.zeros((M,))

for i in range(M):
    for idx, item in enumerate(histories):
        if len(item) <= i:
            continue

        to_plot[i].append(item[i][0])
        temp[i] = item[i][1]
        prob[i].append(np.average(item[i][2]))
        
    #print(prob[0])
        
#print(to_plot)
average = np.zeros((M,))
std = np.zeros((M,))
MIN = np.zeros((M,))

average_p = np.zeros((M,))
std_p = np.zeros((M,))
for idx, item in enumerate(to_plot):
    average[idx] = np.average(item)
    std[idx] = np.std(item)
    MIN[idx] = np.min(item)
    
for idx, item in enumerate(prob):
    average_p[idx] = np.average(item)
    std_p[idx] = np.std(item)
    



In [16]:
z = 1.96
lower_bound = average - z * std
upper_bound = average + z * std
t = np.linspace(1, M, M)

fig, (ax, ax2, ax3) = plt.subplots(3, 1)
ax.plot(t, average, label = 'mean')
ax.fill_between(t, lower_bound, upper_bound, color='gray', alpha=0.2, label='c. interval, 95\%')
ax.grid('on')
#plt.legend()

ax.set_xlabel('iter')
ax.set_ylabel('Fitness function')
ax.set_title(f"Simulated Annealing, Temperature: {T}, decay : {beta}, matrix size: {N}")

ax2.plot(t, temp, label = 'temp')
ax2.set_xlabel('iter')
ax2.set_ylabel('Temperature')

ax3.plot(t, average_p, label = 'prob')


z = 1.96
lower_bound = average_p - z * std_p
upper_bound = average_p + z * std_p


ax3.fill_between(t, lower_bound, upper_bound, color='gray', alpha=0.2, label='c. interval, 95\%')
ax3.set_xlabel('iter')
ax3.set_ylabel('Probability')


#plt.legend()
plt.show()

print(np.min(MIN))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.0


In [17]:

m = 10
N = 9

max_iter = 1600

lst = []
for i in range(m):
    s = GenerateState(N)
    lst.append(s)
    

_,  histories= BeamSearch(lst, max_iter, num_states = m, neighb_num = m)
#print(x)
    




In [18]:
#print(histories)

M = len(histories)

average = np.zeros((M,))
MIN = np.zeros((M,))

std = np.zeros((M,))


for i in range(M):
    average[i] = np.average(histories[i])
    MIN[i] = np.min(histories[i])
    std[i] = np.std(histories[i])


In [19]:
z = 1.96
lower_bound = average - z * std
upper_bound = average + z * std
t = np.linspace(1, M, M)
m = 10
fig, ax = plt.subplots(1, 1)
ax.plot(t, average, label = 'mean')
ax.plot(t, MIN,'r--' ,label = 'minimum')
#ax.fill_between(t, lower_bound, upper_bound, color='gray', alpha=0.2, label='c. interval, 95\%')
ax.grid('on')

ax.set_xlabel('iter')
ax.set_ylabel('Fitness function')
ax.set_title(f"Beam Search, Starting Points: {m}, matrix size: {N}")
#ax.set_ylim([0, 600])

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
m = 30*6
N = 9
elitism = 1
sub_pop = 6
migration_interval = 20
migration_rate = 0.4
rate_increase = 0.05
depth = 100
search_size = 5

pop_size = m


max_iter = 160000//m
k = 5
mutation_rate = 0.004
crossover_prob = 0.65
alpha = 0.05
search_interval = 30

lst = []
for i in range(m):
    s = GenerateState(N)
    lst.append(s)
_,  histories = GenethicAlgorithm(lst, max_iter, pop_size,elitism, k, mutation_rate, alpha, crossover_prob, sub_pop ,migration_interval , migration_rate , rate_increase, depth, search_size, search_interval)



In [21]:
#print(histories)

M = len(histories)

average = np.zeros((M,))
median = np.zeros((M,))
MIN = np.zeros((M,))

std = np.zeros((M,))


for i in range(M):
    average[i] = np.average(histories[i])
    median[i] = np.percentile(histories[i], 20)
    MIN[i] = np.min(histories[i])
    std[i] = np.std(histories[i])


In [22]:
z = 1.96
lower_bound = average - z * std
upper_bound = average + z * std
t = np.linspace(1, M, M)

fig, ax = plt.subplots(1, 1)
ax.plot(t, average, label = 'mean')
ax.plot(t, MIN,'r--' ,label = 'minimum')
#ax.plot(t, median, 'k', label = 'median')
#ax.fill_between(t, lower_bound, upper_bound, color='gray', alpha=0.2, label='c. interval, 95\%')
ax.grid('on')

ax.set_xlabel('iter')
ax.set_ylabel('Fitness function')
ax.set_title(f"Genetic Algorithm, Pop Size: {m}, Sub Pop:{sub_pop}, Mig Int: {migration_interval}, Mig rate {migration_rate}, matrix size: {N}")

plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
print(np.min(MIN))

0.0


In [24]:



MONTE_CARLO = 200

f_0 = [[] for _ in range(5)]
histories = [None]*10

for i in range(MONTE_CARLO):
    
    _, f = RandomSearch(160000, N = 9)
    f_0[0].append(f)
    
    
    
    m = 10
    N = 9
    neighb = 50
    max_iter = 800
    idx = 0
    for j in range(m):
        s = GenerateState(N)
        tmp, _ = GreedySearch(s, max_iter, neighb)
        histories[j] = Fitness(tmp)
        idx+=1
    f_0[1].append(np.min(histories))
    
    
    
    m = 10
    N = 9
    neighb = 1
    beta = 0.95
    Mk = 600
    max_iter = 16000
    T = 2

    for j in range(m):
        s = GenerateState(N)
        tmp, _ = SimulatedAnnealing(s, T, Mk, beta , max_iter , neighb)
        histories[j] = Fitness(tmp)
    f_0[2].append(np.min(histories))
    

    m = 10
    N = 9

    max_iter = 1600

    lst = []
    for i in range(m):
        s = GenerateState(N)
        lst.append(s)
    

    


    
    tmp, _  = BeamSearch(lst, max_iter, num_states = m, neighb_num = m)
    f_0[3].append(Fitness(tmp))
    
    m = 30*6
    N = 9
    elitism = 1
    sub_pop = 6
    migration_interval = 20
    migration_rate = 0.4
    rate_increase = 0.05
    depth = 100
    search_size = 5

    pop_size = m


    max_iter = 160000//m
    k = 5
    mutation_rate = 0.004
    crossover_prob = 0.65
    alpha = 0.05
    search_interval = 30

    lst = []
    for i in range(m):
        s = GenerateState(N)
        lst.append(s)
    tmp, _ = GenethicAlgorithm(lst, max_iter, pop_size,elitism, k, mutation_rate, alpha, crossover_prob, sub_pop ,migration_interval , migration_rate , rate_increase, depth, search_size, search_interval)



    f_0[4].append(Fitness(tmp))

KeyboardInterrupt: 

In [25]:
print(i)
average = np.zeros((5,))
std= np.zeros((5,))

for idx, item in enumerate(f_0):
    average[idx] = np.average(item)
    std[idx] = np.std(item)
    
print(average)
print(std)

114
[1.05564035e+03 1.57894737e-01 4.21052632e-01 0.00000000e+00
 9.03508772e-01]
[155.17636413   0.55534006   0.63376814   0.           1.33751077]


In [26]:

average = np.zeros((5,))
std = np.zeros((5,))

for idx, item in enumerate(f_0):
    if item:
        average[idx] = np.average(item)
        std[idx] = np.std(item)
    
print(average)
print(std)

import csv

array3 = ["Slucajna pretraga", "Gramziva pretraga", "Simulirano kaljenje","Pretraga po snopu", "Genetski algoritam"]
# Define the two arrays
array1 = np.round(average, 3)
array2 =  np.round(std, 3)

# Define the filename to save the data to
filename = r'C:\Users\milos\OneDrive\VIII semestar\VI\domaci1\izvestaj\data.csv'

# Open the file for writing
with open(filename, 'w') as csvfile:
    # Define the writer object
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['mean', 'std'])

    # Write the data rows
    for i in range(len(array1)):
        writer.writerow([array1[i], array2[i], array3[i]])

[1.05564035e+03 1.57894737e-01 4.21052632e-01 0.00000000e+00
 9.03508772e-01]
[155.17636413   0.55534006   0.63376814   0.           1.33751077]
